<a href="https://colab.research.google.com/github/priyadharshini13/rag_using_llm_neo4j/blob/main/Neo4jVectorIndex_Jun23.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain openai tiktoken neo4j python-dotenv transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 974.6/974.6 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 327.4/327.4 kB 21.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 38.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 286.8/286.8 kB 22.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.8/321.8 kB 22.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.1/127.1 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.0/145.0 kB 3.1 MB/s eta 0:00:00


In [ ]:
!pip install -U langchain-community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 27.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 6.0 MB/s eta 0:00:00


In [ ]:
!pip install -U langchain_openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 kB 1.7 MB/s eta 0:00:00


In [ ]:
from langchain.text_splitter import CharacterTextSplitter
from langchain.document_loaders import TextLoader
from langchain.text_splitter import TokenTextSplitter
import os
from langchain.graphs import Neo4jGraph
from langchain_openai import OpenAIEmbeddings

In [ ]:
os.environ['OPENAI_API_KEY']=''
# Kishore
# uri="neo4j+s://65f9819f.databases.neo4j.io"
# user="neo4j"
# password="DBy9xxTXKbgV70MVJOa7EUolMTob-VFMZFMMuJvAvcQ"
# Priya
# Old - Until jun 20
# uri="neo4j+s://339f6349.databases.neo4j.io"
# user="neo4j"
# password="yE_7YPc3aju9xzwv--QNpXl9TOwBekLoVdeIB4Zk6TQ"

# New - jun 23/2026
uri="neo4j+s://b6603b16.databases.neo4j.io"
username="neo4j"
password=""

graph = Neo4jGraph(
    url=uri,
    username=username,
    password=password
)

In [ ]:
raw_documents = TextLoader("/content/booktwo.txt").load()
# Define chunking strategy
text_splitter = TokenTextSplitter(chunk_size=1048, chunk_overlap=24)

# Only take the first the raw_documents
documents = text_splitter.split_documents(raw_documents)

# Read the wikipedia article
# raw_documents = WikipediaLoader(query="The Witcher").load()
# # Define chunking strategy
# text_splitter = CharacterTextSplitter.from_tiktoken_encoder(
#     chunk_size=1000, chunk_overlap=20
# )

# # Remove the summary
# for d in documents:
#     del d.metadata["summary"]

In [ ]:
from langchain_community.vectorstores.neo4j_vector import Neo4jVector


In [ ]:
neo4j_db = Neo4jVector.from_documents(
    documents,
    OpenAIEmbeddings(),
    url=uri,
    username=username,
    password=password,
    database="neo4j",  # neo4j by default
    index_name="Book",  # vector by default
    node_label="Pages",  # Chunk by default
    text_node_property="info",  # text by default
    embedding_node_property="vector",  # embedding by default
    create_id_index=True,  # True by default
)

In [ ]:
neo4j_db.query("SHOW CONSTRAINTS")

[{'id': 4,
  'name': 'constraint_7bc77253',
  'type': 'UNIQUENESS',
  'entityType': 'NODE',
  'labelsOrTypes': ['Pages'],
  'properties': ['id'],
  'ownedIndex': 'constraint_7bc77253',
  'propertyType': None}]

In [ ]:
neo4j_db.query(
    """SHOW INDEXES
       YIELD name, type, labelsOrTypes, properties, options
       WHERE type = 'VECTOR'
    """
)

[{'name': 'Book',
  'type': 'VECTOR',
  'labelsOrTypes': ['Pages'],
  'properties': ['vector'],
  'options': {'indexProvider': 'vector-2.0',
   'indexConfig': {'vector.dimensions': 1536,
    'vector.similarity_function': 'cosine'}}}]

In [ ]:

neo4j_db.similarity_search("Who is the author?", k=1)

[Document(page_content='–92, 151\nmusic, 125–27\nnaps, 62\nnegative emotions, 51–55, 104\nNetherlands, 134\nneural routines, 23–27\nneurons, 78\nneuroscience, 12, 14\nnew tasks, 31, 38–39\nnoise, 125–31, 153\nnoise level experiment, 129–30\nnonconscious routines, 20\nomega-3 fatty acids, 109\nPalmer, Samuel, 4–5\npersonalization, 143, 144\nphysical movements, 10–11\nplanning, 34–38\n\n\x0cplants, 142\nPoor Richard’s Almanack, 5\npositive emotions, 55–57, 104–5\npostures, 140–41\n“power poses,” 10, 140, 143\nThe Power of Habit (Duhigg), 21\nproductivity\nof Benjamin Franklin, 4–9\nblood sugar levels and, 109–11\neffects of light on, 131–36\neffects of noise on, 125–31, 153\nexercise and, 99–107\nfood and drinking strategies for increasing, 119–20\nmastering, 15–16\nmind wandering and, 82–92\nself-control and, 46\nstrategies for, 13–15\nproteins, 108–9\npsychology, 12, 14\nRed Bull energy drinks, 118\nsadness, 54\nSan Francisco State University, 25\nsaturated fats, 109\nSchachter, Stanle

In [ ]:

from langchain_openai import ChatOpenAI
from langchain.chains import RetrievalQAWithSourcesChain

chain = RetrievalQAWithSourcesChain.from_chain_type(
    ChatOpenAI(temperature=0),
    chain_type="stuff",
    retriever=neo4j_db.as_retriever()
)

query = "Who is the author?"

chain.invoke(
    {"question": query},
    return_only_outputs=True,
)


{'answer': 'The author is Josh Davis, Ph.D.\n',
 'sources': '/content/booktwo.txt'}

In [ ]:
query = "What are the strategies given in the book?"

chain.invoke(
    {"question": query},
    return_only_outputs=True,
)


{'answer': 'The strategies given in the book are:\n1. Recognize Your Decision Points\n2. Manage Your Mental Energy\n3. Stop Fighting Distractions\n4. Leverage Your Mind–Body Connection\n5. Make Your Workspace Work for You\n',
 'sources': '/content/booktwo.txt'}

In [ ]:
query = "What are the strategies given in the book?"
result = chain.invoke(
    {"question": query},
    return_only_outputs=True,
)
print(result)


{'answer': 'The strategies given in the book are:\n1. Recognize Your Decision Points\n2. Manage Your Mental Energy\n3. Stop Fighting Distractions\n4. Leverage Your Mind–Body Connection\n5. Make Your Workspace Work for You\n', 'sources': '/content/booktwo.txt'}


In [ ]:
query = "Whose story is given as an example and what special about that person?"
result = chain.invoke(
    {"question": query},
    return_only_outputs=True,
)
print(result)


{'answer': 'The story given as an example is about Benjamin Franklin. What is special about him is that he was a model of effectiveness and productivity, with a list of accomplishments including author, inventor, scientist, printer, philosopher, politician, postmaster, diplomat, and more. He was known for his work ethic, cleverness, and ability to outperform others. Additionally, he spent time on hobbies, socializing, and pursuing interests outside of his primary profession, which ultimately contributed to his success.\n', 'sources': '/content/booktwo.txt'}
